In [42]:
import torch
import numpy as np

# operate tensor

## view

**view** only operate the calulate shape of tensor but **not** change the **address**, so the value are co-change

In [23]:
x=torch.randn(4)
y=x.view(2,2)
x

tensor([ 1.0196,  0.5737,  1.4164, -1.0359])

In [24]:
x+=1
y

tensor([[ 2.0196,  1.5737],
        [ 2.4164, -0.0359]])

## transform to tensor<br/>
**torch.tensor() ** will **copy** the origin data<br/>
**torch.from_numpy()** only copy the address

In [63]:
a=np.array([3,4,5])
b=torch.tensor(a)
a+=1
b

tensor([3, 4, 5], dtype=torch.int32)

In [64]:
a=np.array([3,4,5])
c=torch.from_numpy(a)
a+=1
c

tensor([4, 5, 6], dtype=torch.int32)

## RAM cost
if we use <br/>
a=a+1<br/>we will get another address to contain the value of a+1<br/>
increase rom cost

In [67]:
x=np.array([2,3,4])
id_before=id(x)
x=x+1
id(x)==id_before

False

## broadcast mechanism
$$one is :3\times 4\times5\\
 the\ other: 4\times 6 \\
we\ get 3\times 4\times6\ according\ to\ mechanism$$

## cuda

In [25]:
if torch.cuda.is_available():
    device=torch.device('cuda')
    y=torch.ones_like(x,device=device)
    x=x.to(device)
    z=x+y
    z.to("cpu",dtype=torch.float)

## backward propagation

leaf node has **None** grad_fn

In [70]:
x=torch.tensor([3.,4.,5.],requires_grad=True)
y=x+4
print(x.grad_fn,y.grad_fn)

None <AddBackward0 object at 0x000002095B7E8390>


backward propagate will accumulate before gradiation, so we need to reset the grad

In [71]:
out=x.mean()
out.backward()
out2=x.sum()
x.grad.data.zero_()
out2.backward()
x.grad

tensor([1., 1., 1.])

when the outcome is a scalar, we pass no parameter to **backward**

we can not computer gradient to a tensor reason is that it's not effient to compute tensor to tensor gradient especially for high dimension. ig 
$$X=m\times n \ and\ Y=q \times p$$ we will get $$\partial Y/\partial X\ is \ m\times n \times q\times p$$

what we gonna do is to pass a scalar taht is the same shape with outcome to **backward()**

In [72]:
x = torch.tensor([1.0, 2.0, 3.0, 4.0], requires_grad=True)
y = 2 * x
z = y.view(2, 2)
print(z)

tensor([[2., 4.],
        [6., 8.]], grad_fn=<ViewBackward>)


In [73]:
v = torch.tensor([[1.0, 0.1], [0.01, 0.001]], dtype=torch.float)
z.backward(v)
print(x.grad)

tensor([2.0000, 0.2000, 0.0200, 0.0020])


## interrupt grad track

In [74]:
x = torch.tensor(1.0, requires_grad=True)
y1 = x ** 2
with torch.no_grad():
 y2 = x ** 3
y3 = y1 + y2

print(x.requires_grad)
print(y1, y1.requires_grad) # True
print(y2, y2.requires_grad) # False
print(y3, y3.requires_grad) # True

True
tensor(1., grad_fn=<PowBackward0>) True
tensor(1.) False
tensor(2., grad_fn=<AddBackward0>) True


compute using **x.data** can seperate from the computation graph